In [1]:
import requests
import xml.etree.ElementTree as ET

from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.preprocessors import DocumentSplitter

# Get sitemap

In [2]:
sitemap_url = 'https://www.eak.admin.ch/eak/de/home.sitemap.xml'

try:
    response = requests.get(sitemap_url)
    response.raise_for_status()

except requests.exceptions.RequestException as e:
    print(f"Error fetching the sitemap: {e}")

# Extract sitemap URLs to scrap

In [3]:
try:
    # Parse the XML sitemap
    tree = ET.ElementTree(ET.fromstring(response.content))
    root = tree.getroot()

    # Define the namespace dictionary
    namespaces = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

    # List to store all URLs
    url_list = []

    # Iterate over all 'url' elements and extract the 'loc' URL
    for url_element in root.findall('ns:url', namespaces):
        loc_element = url_element.find('ns:loc', namespaces)
        if loc_element is not None and loc_element.text:
            url_list.append(loc_element.text)

except ET.ParseError as e:
    print(f"Error parsing the XML file: {e}")
except FileNotFoundError as e:
    print(f"File not found: {e}")

In [4]:
len(url_list)

145

In [5]:
url_list

['https://www.eak.admin.ch/eak/de/home.html',
 'https://www.eak.admin.ch/eak/de/home/sitemap.html',
 'https://www.eak.admin.ch/eak/de/home/suche.html',
 'https://www.eak.admin.ch/eak/de/home/404.html',
 'https://www.eak.admin.ch/eak/de/home/Firmen.html',
 'https://www.eak.admin.ch/eak/de/home/Firmen/Anschluss.html',
 'https://www.eak.admin.ch/eak/de/home/Firmen/Anschluss/anschlusspflicht.html',
 'https://www.eak.admin.ch/eak/de/home/Firmen/Anschluss/zweigniederlassungen.html',
 'https://www.eak.admin.ch/eak/de/home/Firmen/Anschluss/anschlussverfahren.html',
 'https://www.eak.admin.ch/eak/de/home/Firmen/Anschluss/arbeitgeberkontrolle.html',
 'https://www.eak.admin.ch/eak/de/home/Firmen/Personal.html',
 'https://www.eak.admin.ch/eak/de/home/Firmen/Personal/anmeldung-von-arbeitnehmern.html',
 'https://www.eak.admin.ch/eak/de/home/Firmen/Personal/abmeldung-von-arbeitnehmern.html',
 'https://www.eak.admin.ch/eak/de/home/Firmen/Personal/versicherungsausweis.html',
 'https://www.eak.admin.ch/

# Fetch content with haystack LinkContentFetcher()

In [6]:
fetcher = LinkContentFetcher()

In [7]:
# Only scrap n webpages for testing
scrap_n = 5

streams = fetcher.run(urls=url_list[:scrap_n])
content = streams["streams"]

In [8]:
len(content)

5

# Convert webpage content to Document with haystack HTMLToDocument()

In [9]:
converter = HTMLToDocument()

In [10]:
docs = converter.run(sources=content)["documents"]
docs

[Document(id=1ca3a67bb25dd21955b637c4848169627bd3dbde960de1aba360a63d6f1421a3, content: 'Die Eidgenössische Ausgleichskasse EAK mit Sitz in Bern ist eine Abteilung der Zentralen Ausgleichss...', meta: {'content_type': 'text/html', 'url': 'https://www.eak.admin.ch/eak/de/home.html'}),
 Document(id=78a87134c941a811c5a3d66397f5c7451349a970f2fef2d2585b1c78c65d0362, content: 'Firmen
 - Anschluss
 - Personal
 - Externe Aufträge
 - Arbeiten im Ausland
 - Beiträge und Löhne
 - Famili...', meta: {'content_type': 'text/html', 'url': 'https://www.eak.admin.ch/eak/de/home/sitemap.html'}),
 Document(id=036f362bbd3f39ea6ae40095258572779379e3a27593547305ee55d5313cd49e, content: 'Navigation
 Zur Ausführung der Suche wird Java Script benötigt.
 0 Ergebnisse gefunden
 Zum Seitenanfan...', meta: {'content_type': 'text/html', 'url': 'https://www.eak.admin.ch/eak/de/home/suche.html'}),
 Document(id=fa22522ab1a7207a86cfd9d78d15e2378510dffd05132ea579830e4d58265aef, content: 'Die aufgerufene Seite wurde leide

In [11]:
# Get URL
docs[0].meta["url"]

'https://www.eak.admin.ch/eak/de/home.html'

# Clean documents with haystack DocumentCleaner()

In [12]:
cleaner = DocumentCleaner(
	remove_empty_lines=True,
	remove_extra_whitespaces=True,
	remove_repeated_substrings=False)

In [13]:
docs = cleaner.run(documents=docs)
docs

{'documents': [Document(id=1ca3a67bb25dd21955b637c4848169627bd3dbde960de1aba360a63d6f1421a3, content: 'Die Eidgenössische Ausgleichskasse EAK mit Sitz in Bern ist eine Abteilung der Zentralen Ausgleichss...', meta: {'content_type': 'text/html', 'url': 'https://www.eak.admin.ch/eak/de/home.html'}),
  Document(id=78a87134c941a811c5a3d66397f5c7451349a970f2fef2d2585b1c78c65d0362, content: 'Firmen
  - Anschluss
  - Personal
  - Externe Aufträge
  - Arbeiten im Ausland
  - Beiträge und Löhne
  - Famili...', meta: {'content_type': 'text/html', 'url': 'https://www.eak.admin.ch/eak/de/home/sitemap.html'}),
  Document(id=036f362bbd3f39ea6ae40095258572779379e3a27593547305ee55d5313cd49e, content: 'Navigation
  Zur Ausführung der Suche wird Java Script benötigt.
  0 Ergebnisse gefunden
  Zum Seitenanfan...', meta: {'content_type': 'text/html', 'url': 'https://www.eak.admin.ch/eak/de/home/suche.html'}),
  Document(id=fa22522ab1a7207a86cfd9d78d15e2378510dffd05132ea579830e4d58265aef, content: 'Die auf

# Split documents with haystack DocumentSplitter()

In [14]:
splitter = DocumentSplitter(
    split_by="passage",
    split_length=1,
    split_overlap=0)

In [16]:
docs = splitter.run(documents=docs["documents"])

In [17]:
docs

{'documents': [Document(id=e8d2e97260140cb766b3d26d57c83ef7431d745a8f1d409f0442f6a420d8f053, content: 'Die Eidgenössische Ausgleichskasse EAK mit Sitz in Bern ist eine Abteilung der Zentralen Ausgleichss...', meta: {'content_type': 'text/html', 'url': 'https://www.eak.admin.ch/eak/de/home.html', 'source_id': '1ca3a67bb25dd21955b637c4848169627bd3dbde960de1aba360a63d6f1421a3', 'page_number': 1}),
  Document(id=6bb7eda01749087345ca0d253bc674403e4c56787f567a81e0180e375661e992, content: 'Firmen
  - Anschluss
  - Personal
  - Externe Aufträge
  - Arbeiten im Ausland
  - Beiträge und Löhne
  - Famili...', meta: {'content_type': 'text/html', 'url': 'https://www.eak.admin.ch/eak/de/home/sitemap.html', 'source_id': '78a87134c941a811c5a3d66397f5c7451349a970f2fef2d2585b1c78c65d0362', 'page_number': 1}),
  Document(id=ef5ed7b42f0ea2b04ac61c9a9f4b225d791028f4cfd1606b7a51c8dd74fe9ab5, content: 'Navigation
  Zur Ausführung der Suche wird Java Script benötigt.
  0 Ergebnisse gefunden
  Zum Seitenanfan.

In [19]:
len(docs["documents"])

5